In [1]:
import torch

# 🧠 PyTorch 自动求导机制要点笔记

## 1. 创建叶子节点（Leaf Node）张量
- 使用 `requires_grad=True` 指定是否记录对该张量的操作；
- 默认为 `False`；
- 如果后续涉及依赖操作，PyTorch 会自动将其设为 `True`。

---

## 2. 控制是否记录梯度
- 使用 `.requires_grad_()` 方法控制；
- 使用 `.detach()` 或 `with torch.no_grad()` 可停止梯度记录（不追踪历史）；
- 常用于模型评估和测试阶段。

---

## 3. 运算生成的张量（非叶子节点）
- 会自动带有 `grad_fn` 属性，表示它的梯度函数；
- 叶子节点的 `grad_fn` 为 `None`。

---

## 4. 调用 `.backward()` 方法
- 会从输出张量开始反向传播，自动计算梯度；
- 梯度将累加到 `.grad` 属性中；
- 计算完成后，非叶子节点的梯度会被释放。

---

## 5. `.backward()` 参数要求
- 输入参数应和调用 `.backward()` 的张量维度相同；
- 支持 broadcast；
- 如果张量是标量（0 维），则参数可省略。

---

## 6. 多次反向传播需注意
- 中间缓存默认会被释放；
- 如果需要多次反向传播，请设置 `retain_graph=True`；
- 每次反向传播时，梯度是累加的。

---

## 7. `.backward()` 后梯度默认清空
- 非叶子节点的 `.grad` 会被清空；
- 如需保留，请手动保存或 `retain_graph=True`。

---

## 8. `no_grad` 语境控制
- 使用 `torch.no_grad()` 或设置 `y.requires_grad=False` 可显式关闭追踪；
- 在推理阶段广泛使用，可节省内存与计算。

---

## 🔁 计算图结构说明
- PyTorch 构建的计算图是**动态图**，每次前向传播时重建；
- 与 TensorFlow 的静态图不同；
- 整个图结构是**有向无环图（DAG）**。

In [2]:
x = torch.tensor([1.0, 2.0], requires_grad=True)
print(x.requires_grad)  # True

True


In [3]:
y = x.detach()
print(y.requires_grad)  # False

with torch.no_grad():
    z = x * 2
print(z.requires_grad)  # False

False
False


In [4]:
z = x * 3 + 1
print(z.grad_fn)  # 有 grad_fn

In [5]:
out = z.sum()
out.backward()
print(x.grad)  # x 的梯度

tensor([3., 3.])


In [6]:
x = torch.tensor([[1., 2.], [3., 4.]], requires_grad=True)
y = x ** 2
grad_output = torch.ones_like(y)
y.backward(grad_output)  # 必须传参，因为 y 不是标量
print(x.grad)

tensor([[2., 4.],
        [6., 8.]])


In [7]:
x = torch.tensor([1., 2., 3.], requires_grad=True)
y = x * 2
y.sum().backward(retain_graph=True)
print(x.grad) # tensor([2., 2., 2.])
y.sum().backward()  # 第二次传播需保留图
print(x.grad) # tensor([4., 4., 4.])

tensor([2., 2., 2.])
tensor([4., 4., 4.])


In [8]:
a = torch.tensor([2.0], requires_grad=True)
b = a * 3  # b 是非叶子节点
loss = b.sum()
loss.backward()
print(b.grad)  # None

None


/tmp/ipykernel_614/2864770302.py:5: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at aten/src/ATen/core/TensorBody.h:489.)
  print(b.grad)  # None


In [9]:
with torch.no_grad():
    z = x * 3
print(z.requires_grad)  # False

False


In [10]:
# 创建叶子节点
x = torch.tensor([2.0], requires_grad=True)

# 生成非叶子节点
y = x * 3
y.retain_grad()  # ⭐ 保留中间变量的梯度

# 计算结果
z = y ** 2
z.backward()

# 叶子节点的梯度
print("x.grad:", x.grad)  # ✅ 有梯度

# 非叶子节点的梯度
print("y.grad:", y.grad)  # ✅ 有梯度，因为调用了 retain_grad()

x.grad: tensor([36.])
y.grad: tensor([12.])


In [11]:
# 1. 创建张量，设置 requires_grad=True 才能参与自动求导
x = torch.tensor([2.0], requires_grad=True)

# 2. 定义函数 y = x^3
y = x ** 3

# 3. 求一阶导数 dy/dx = 3x^2
dy_dx = torch.autograd.grad(outputs=y, inputs=x, create_graph=True)[0]
print("dy/dx:", dy_dx)  # 输出: tensor([12.])

# 4. 对一阶导再求导（二阶导数）d²y/dx² = 6x
d2y_dx2 = torch.autograd.grad(outputs=dy_dx, inputs=x)[0]
print("d²y/dx²:", d2y_dx2)  # 输出: tensor([6.])

dy/dx: tensor([12.], grad_fn=<MulBackward0>)
d²y/dx²: tensor([12.])
